In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from sklearn.model_selection import KFold
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

from pyastrotrader import calculate_chart, calculate_aspects, calculate_transits
from pyastrotrader.utils import create_input_json
from pyastrotrader.constants import *

In [ ]:
SWING_TRADE_DURATION = 5
SWING_EXPECTED_VOLATILITY = 5
STAGNATION_THRESHOLD = 3
SOURCE_FILE = "./input/PETR4_Daily"

In [ ]:
StockPrices = pd.read_csv("{}.csv".format(SOURCE_FILE))
StockPrices['Counter'] = np.arange(len(StockPrices))

In [ ]:
def correct_date(x):
    date_str = str(x['Date'])
    return date_str[0:4] + '-' + date_str[4:6] + '-' + date_str[6:8]    

def change_sign(x,y):
    return  not ((x > 0 and y > 0) or (x < 0 and y < 0))

In [ ]:
def get_previous_stock_price(df, x):
    if x['Counter'] > (SWING_TRADE_DURATION - 1):
        return float(df[df['Counter'] == (x['Counter'] - SWING_TRADE_DURATION)]['Price'])
    else:
        return 0

def get_future_stock_price(df, x, max):
    if x['Counter'] < (max - SWING_TRADE_DURATION):
        return float(df[df['Counter'] == (x['Counter'] + SWING_TRADE_DURATION)]['Price'])
    else:
        return 0    
    
def calculate_current_trend(x):
    if x['PreviousStartPrice'] > 0:
        return ((float(x['Price']) / float(x['PreviousStartPrice'])) - 1) * 100
    else:
        return 0;

def get_previous_stock_date(df, x):
    if x['Counter'] > (SWING_TRADE_DURATION - 1):
        return float(df[df['Counter'] == (x['Counter'] - SWING_TRADE_DURATION)]['Date'])
    else:
        return 0

def get_future_stock_date(df, x, max):
    if x['Counter'] < (max - SWING_TRADE_DURATION):
        return float(df[df['Counter'] == (x['Counter'] + SWING_TRADE_DURATION)]['Date'])
    else:
        return 0    

def calculate_future_trend(x):
    if x['FutureFinalPrice'] > 0:
        return ((float(x['FutureFinalPrice']) / float(x['Price'])) - 1) * 100
    else:
        return 0;

def calculate_intraday_volatility(df, x):
    current_range = df[df['Counter'] >= (x['Counter'] - (SWING_TRADE_DURATION * 2))]
    current_range = current_range[current_range['Counter'] < x['Counter']]
    return (((current_range['High'] / current_range['Low']) - 1).sum() / (SWING_TRADE_DURATION * 2)) * 100

In [ ]:
def calculate_swing_strenght(x):
    if float(x['CurrentTrend']) != 0 and float(x['FutureTrend'] != 0):
        return abs(float(x['CurrentTrend']) - float(x['FutureTrend']))
    else:
        return 0

def is_swing_trade(x):
    return 1 if change_sign(x['FutureTrend'], x['CurrentTrend']) and \
                x['SwingStrength'] > (x['IntradayVolatility'] * SWING_EXPECTED_VOLATILITY) and \
                abs(x['FutureTrend']) >= abs(x['CurrentTrend']) else 0

def clean_swing_trade(df, x):
    if x['IsSwing'] == 1:
        current_range = df[df['Counter'] <= (x['Counter'] + SWING_TRADE_DURATION)]
        current_range = current_range[current_range['Counter'] > x['Counter']]
        if current_range['IsSwing'].sum() > 0:
            return 0
        else:
            return 1
    else:
        return 0
    
def detect_price_increase(x):
    return 1 if abs(x['FutureTrend']) >= (x['IntradayVolatility'] * STAGNATION_THRESHOLD) and x['FutureTrend'] > 0 else 0

def detect_price_decrease(x):
    return 1 if abs(x['FutureTrend']) >= (x['IntradayVolatility'] * STAGNATION_THRESHOLD) and x['FutureTrend'] < 0 else 0

def detect_price_stagnated(x):
    return 0 if abs(x['FutureTrend']) >= (x['IntradayVolatility'] * STAGNATION_THRESHOLD) else 1

In [ ]:
max_counter = StockPrices['Counter'].max()

StockPrices['CorrectedDate'] = StockPrices.apply( lambda x :  correct_date(x), axis =1 )
StockPrices['PreviousStartPrice'] = StockPrices.apply( lambda x :  get_previous_stock_price(StockPrices, x), axis =1 )
StockPrices['FutureFinalPrice'] = StockPrices.apply( lambda x :  get_future_stock_price(StockPrices, x, max_counter), axis =1 )
StockPrices['PreviousStartDate'] = StockPrices.apply( lambda x :  get_previous_stock_date(StockPrices, x), axis =1 )
StockPrices['FutureFinalDate'] = StockPrices.apply( lambda x :  get_future_stock_date(StockPrices, x, max_counter), axis =1 )

StockPrices['CurrentTrend'] = StockPrices.apply(lambda x : calculate_current_trend(x), axis = 1)

StockPrices['FutureTrend'] = StockPrices.apply(lambda x : calculate_future_trend(x), axis = 1)
StockPrices['SwingStrength'] = StockPrices.apply(lambda x: calculate_swing_strenght(x), axis =1)
StockPrices['IntradayVolatility'] = StockPrices.apply(lambda x: calculate_intraday_volatility(StockPrices, x), axis =1)

StockPrices['IsSwing'] = StockPrices.apply(lambda x: is_swing_trade(x), axis =1)
StockPrices['IsSwing'] = StockPrices.apply(lambda x:clean_swing_trade(StockPrices, x), axis =1)

StockPrices['StockIncreasedPrice'] = StockPrices.apply(lambda x:detect_price_increase(x), axis =1)
StockPrices['StockDecreasedPrice'] = StockPrices.apply(lambda x:detect_price_decrease(x), axis =1)
StockPrices['StockStagnated'] = StockPrices.apply(lambda x:detect_price_stagnated(x), axis =1)

StockPrices['TARGET_IS_SWING'] = StockPrices['IsSwing']

In [ ]:
#StockPrices[StockPrices['IsSwing'] == 1].sort_values(by=['Date'], ascending=[0]).head(20)

In [ ]:
#StockPrices[StockPrices['StockIncreasedPrice'] == 1].sort_values(by=['Date'], ascending=[0]).head(20)

In [ ]:
#StockPrices[StockPrices['StockDecreasedPrice'] == 1].sort_values(by=['Date'], ascending=[0]).head(20)

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=StockPrices['CorrectedDate'],
                open=StockPrices['Open'],
                high=StockPrices['High'],
                low=StockPrices['Low'],
                close=StockPrices['Price'])])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
petr4_natal_chart_input = create_input_json('1953-10-03T19:05:00-03:00', 
                                      './config/default_parameters.json', 
                                      './config/default_config.json')

petr4_natal_chart = calculate_chart(petr4_natal_chart_input)
dates_to_generate = list(StockPrices['CorrectedDate'])

planets = [SUN,MOON,SATURN]
aspects = [CONJUNCTION, SQUARE, TRINE, OPPOSITION]

charts = {}
transits = {}

for current_date in dates_to_generate:
    chart_input = create_input_json(current_date + 'T10:00:00-03:00', 
                                      './config/default_parameters.json', 
                                      './config/default_config.json')
    charts[current_date] = calculate_chart(chart_input)
    transits[current_date] = calculate_transits(petr4_natal_chart, charts[current_date], planets, aspects, 4)    

In [ ]:
def is_aspected(row, first_planet, second_planet, aspect):
    c_transits = transits[row['CorrectedDate']]
    found_params = [x for x in c_transits if (x['c_planet'] == first_planet and x['n_planet'] == second_planet and x['c_aspect'] == aspect) or \
                                           (x['n_planet'] == first_planet and x['c_planet'] == second_planet and x['c_aspect'] == aspect) ]
    return 1 if len(found_params) > 0 else 0

In [ ]:
astro_columns = []

for first_planet in planets:
    for second_planet in planets:
        for aspect in aspects:
            column_name="ASTRO_{}_{}_{}".format(PLANETS[first_planet],ASPECT_NAME[aspect],PLANETS[second_planet]).upper()
            astro_columns.append(column_name)
            StockPrices['StockIncreasedPrice'] = StockPrices.apply(lambda x:detect_price_increase(x), axis =1)
            StockPrices[column_name] = StockPrices.apply(lambda x:is_aspected(x, first_planet, second_planet, aspect), axis =1)

In [ ]:
output_excel_file='./output/PETR4.Analisys.xlsx'
StockPrices.to_excel(output_excel_file)

In [ ]:
def create_booster_swing_trade(eta,depth,num_trees, train_x, train_y, test_x, test_y, columns):
    param = {}
    param['booster'] = 'gbtree'
    param['eta'] = eta
    param['objective'] = 'binary:logistic'
    param['eval_metric'] = 'auc'
    param['tree_method'] = 'auto'
    param['silent'] = 0
    param['max_depth'] = depth
    param['subsample'] = 0.5
    num_round = num_trees
    dtrain = xgb.DMatrix(train_x, train_y, feature_names = columns)
    dtest = xgb.DMatrix(test_x, test_y, feature_names = columns)
    train_labels = dtrain.get_label()
    ratio = float(np.sum(train_labels == 0)) / np.sum(train_labels == 1) 
    param['scale_pos_weight'] = ratio
    gpu_res = {}
    booster = xgb.train(param, dtrain, num_round, evals_result=gpu_res, evals = [])    
    return booster, dtrain, dtest

In [ ]:
X = StockPrices[astro_columns].values
Y = StockPrices['TARGET_IS_SWING'].values

booster, dtrain, dtest = create_booster_swing_trade(0.3, 7, 30, X, Y, X, Y, astro_columns)
relevant_features = sorted( ((v,k) for k,v in booster.get_score().items()), reverse=True)
relevant_features